In [ ]:
import socket
import struct
import time
from datetime import datetime, timedelta

import requests
import json

NTP_SERVER = "pool.ntp.org"
TIMEZONE_OFFSET = 7  # GMT+7
CHECK_TIMES = ["11:34", "18:00"]

def get_ntp_time():
    client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    client.settimeout(5)

    ntp_packet = b'\x1b' + 47 * b'\0'
    client.sendto(ntp_packet, (NTP_SERVER, 123))
    data, _ = client.recvfrom(1024)

    if data:
        ntp_time = struct.unpack("!12I", data)[10] - 2208988800  # แปลงจาก NTP timestamp เป็น Unix timestamp
        utc_time = datetime.utcfromtimestamp(ntp_time)
        local_time = utc_time + timedelta(hours=TIMEZONE_OFFSET)
        return local_time
    return None

# Step 1: Get token
def get_token():
    url = "https://api.dcrth.co.th/api/v1/auth/login"
    payload = {
        "username": "gsb_master",
        "password": "gsb_master"
    }
    headers = {
        "content-type": "application/json"
    }
    response = requests.post(url, headers=headers, json=payload).json()
    #json_text = ""
    #json_text = "'" + response.text + "'"
    #print("response = ")
    #print(response['data']['token'])
    json_token = response['data']['token']
    #return response.json().get("token")
    return json_token


# Step 2: Get device list
def get_device_list(token):
    print("token of get device = " + token)
    url = "https://api.dcrth.co.th/api/v1/device/list?page=1&limit=1"
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {token}"
    }

    #print("header = ")
    #print(headers)
    get_response = requests.get(url, headers=headers).json()
    #response.raise_for_status()
    #print("device list = ")
    #print(get_response)
    return get_response

# Step 3: Analyze JSON data
def analyze_device_list(device_list):
    status_count = {0: 0, 1: 0, 2: 0}
    alarm_count = 0
    for device in device_list['data']['data']:
        serial_number = device.get("serial_number", "")
        if "FAAC-9001" <= serial_number <= "FAAC-9050":
            status = device.get("status")
            if status in status_count:
                status_count[status] += 1
            if device.get("alarm") == 1:
                alarm_count += 1
    return status_count, alarm_count

# Step 4: Send data to Telegram
def send_to_telegram(message):
    telegram_url1 = "https://api.telegram.org/bot7839341791:AAEXC5NTO2xjPY0bcPvs4LX2vMLpNy7OdXs/sendMessage?chat_id=7860881906&text=" + message
    telegram_url2 = "https://api.telegram.org/bot7839341791:AAEXC5NTO2xjPY0bcPvs4LX2vMLpNy7OdXs/sendMessage?chat_id=-4749718153&text=" + message

    #print(telegram_url)
    requests.get(telegram_url1)
    requests.get(telegram_url2)

# Main execution
def main_process():
    token = get_token()
    #print("token = ")
    #print(token)
    device_list = get_device_list(token)
    status_count, alarm_count = analyze_device_list(device_list)
    message = (
        f"Status 0 count: {status_count[0]}\n"
        f"Status 1 count: {status_count[1]}\n"
        f"Status 2 count: {status_count[2]}\n"
        f"Alarm count: {alarm_count}"
    )

    telegram_message = (
          f"     \U0001F4DD     รายงานสถานะประจำวัน %0A     \U00002705     Normal : {status_count[0]} จุด%0A"
          f"     \U000026A0     Trouble : {status_count[1]} จุด%0A"
          f"     \U000026AA     Off-line : {status_count[2]} จุด%0A"
          f"     \U0001F6A8     Alarm : {alarm_count} จุด"
        )

    #print(telegram_message)
    send_to_telegram(telegram_message)

while True:
            main_process()
            time.sleep(60)  # เช็คทุก 60 วินาที
            break